In [ ]:
cocci = ''.join(open(os.path.abspath('./patch.cocci')).readlines())

def modify_src(source, verbose=False):
    if verbose:
        pprint(source)
    patch = cocci
    f = open('.temp.cocci', 'w')
    f.write(''.join(patch))
    f.close()
    execute = '/usr/local/bin/spatch -sp_file .temp.cocci -in_place %s >/dev/null 2>/dev/null' % source
    if verbose:
        execute = '/usr/local/bin/spatch -sp_file .temp.cocci %s' % source
    os.system(execute)
    
modify_src(os.path.abspath('./json_tokener.c'), verbose=True)

In [ ]:
import json, os, sys
from pprint import pprint

print("cwd = %s" % os.getcwd())